# Aprendizado Zero-Shot
- Cenário onde o modelo de IA é treinado para reconhecer e categorizar conceitos ou objetos sem nunca ter visto um exemplo dessas categorias ou conceitos com antecedência
- Ele não se refere a um algoritmo específico mas à natureza do próprio problema
- Os problemas de aprendizado Zero-Shot fazem o uso de informações auxiliares como descrições textuais, atributos, representações incorporadas ou outras informações semânticas relevantes para a tarefa em questão
- A transferência de aprendizado também é muito utilizada em problemas desse tipo, utilizar um modelo já treinado para uma nova tarefa pode ser útil para minimizar o tempo e recursos necessários

In [1]:
import torch
from datasets import load_dataset
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForCausalLM,
)
from sentence_transformers import SentenceTransformer
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    confusion_matrix,
    classification_report,
)
import numpy as np
import random
from tqdm.auto import tqdm

# Reprodutibilidade
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

device = "cuda" if torch.cuda.is_available() else "cpu"
device


c:\Users\Mateus\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'cpu'

In [2]:
# ============================
# 3.1 Carregar dataset IMDb
# ============================
dataset = load_dataset("stanfordnlp/imdb")

# Subconjuntos para caber no Colab
N_TRAIN = 2000
N_TEST  = 200

train_data = dataset["train"].shuffle(seed=SEED).select(range(N_TRAIN))
test_data  = dataset["test"].shuffle(seed=SEED).select(range(N_TEST))

label_map = {0: "negative", 1: "positive"}

print("Exemplo de dado de treino:")
print(train_data[0])
print("Tamanhos -> train:", len(train_data), "| test:", len(test_data))

Exemplo de dado de treino:
{'text': 'There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier\'s plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it\'s the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...', 'label': 1}
Tamanhos -> train: 2000 | test: 200


In [ ]:
#Modelo bart-large-mnli é um dos modelos mais eficazes do mundo para ZERO-SHOT
#Treinado justamente em NLI — a tarefa que permite fazer classificação sem treino

# ============================
# 3.2 Baseline: Zero-Shot com BART-MNLI
# ============================
zero_shot_classifier = pipeline(
    "zero-shot-classification",
    model="facebook/bart-large-mnli",
    device=0 if device == "cuda" else -1,
)

candidate_labels = ["positive", "negative"]

def predict_zero_shot(text: str) -> str:
    result = zero_shot_classifier(
        text,
        candidate_labels=candidate_labels,
        multi_label=False,
    )
    # Label com maior score
    return result["labels"][0]

c:\Users\Mateus\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Mateus\.cache\huggingface\hub\models--facebook--bart-large-mnli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Device set to use cpu


In [4]:
y_true = []
y_pred_zero_shot = []

for sample in tqdm(test_data, desc="Baseline Zero-Shot (BART-MNLI)"):
    text = sample["text"]
    true_label = label_map[sample["label"]]
    pred_label = predict_zero_shot(text)
    y_true.append(true_label)
    y_pred_zero_shot.append(pred_label)

print("=== Métricas Baseline: BART Zero-Shot ===")
print(classification_report(y_true, y_pred_zero_shot, digits=4))
print("Matriz de confusão:")
print(confusion_matrix(y_true, y_pred_zero_shot))

Baseline Zero-Shot (BART-MNLI): 100%|██████████| 200/200 [05:41<00:00,  1.71s/it]

=== Métricas Baseline: BART Zero-Shot ===
              precision    recall  f1-score   support

    negative     0.8785    0.9038    0.8910       104
    positive     0.8925    0.8646    0.8783        96

    accuracy                         0.8850       200
   macro avg     0.8855    0.8842    0.8847       200
weighted avg     0.8852    0.8850    0.8849       200

Matriz de confusão:
[[94 10]
 [13 83]]


# Por que o modelo Zero-Shot foi melhor que o RAG?

- Motivo 1: RAG não é adequado para tarefas simples de classificação, sendo melhor para 
    - recuperar fatos
    - responder perguntas
- Motivo 2: Aumentar a base de dados pode melhorar o modelo
- Motivo 3: A tarefa não depende de conhecimento externo, o texto já contem toda informação necessária
- Motivo 4: O contexto aumenta o nível de “prompt noise”
    - mais tokens inúteis o modelo precisa processar
    - maior a chance de se desviar do foco